In [1]:
import pandas as pd
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import folium
pd.options.display.max_columns = None
pd.options.display.max_rows = None



In [2]:
stadtteile = pd.read_html('https://de.wikipedia.org/wiki/Liste_der_Stadtteile_M%C3%BCnchens')[0][['Stadtteil']]

In [3]:
stadtteile.drop([22],inplace=True)
stadtteile.loc[18] = 'Obergiesing'
stadtteile.loc[19] = 'Untergiesing'
stadtteile.loc[21] = 'Holzapfelkreuth'
stadtteile.loc[44] = 'Schwabing'
stadtteile.loc[45] = 'Nord Schwabing'
stadtteile.loc[47] = 'Sendling'
stadtteile.loc[48] = 'Untersendling'
stadtteile.loc[49] = 'Obersendling'


In [4]:
stadtteile = stadtteile.append(pd.DataFrame({'Stadtteil':['Altschwabing','West Schwabing','Mittersendling']}),ignore_index=True).sort_values(by='Stadtteil').reset_index(drop=True)

In [5]:
address = 'Holzapfelkreuth Munich, DE'
location = Nominatim(user_agent="toronto").geocode(address)
longitude = location.longitude
latitude = location.latitude

for stadtteil in stadtteile['Stadtteil']:
    print(stadtteil)
    address = str(stadtteil)+' Munich, DE'
    location = Nominatim(user_agent="toronto").geocode(address)
    stadtteile.loc[stadtteile['Stadtteil'] == stadtteil,'Latitude'] = location.latitude
    stadtteile.loc[stadtteile['Stadtteil'] == stadtteil,'Longitude'] = location.longitude

stadtteile.to_csv('stadtteile_muc.csv')

In [26]:
stadtteile = pd.read_csv('stadtteile_muc.csv')

In [27]:
map_city = folium.Map(location=[latitude, longitude], zoom_start=14)
map_city

In [9]:
CLIENT_ID = 'WC4YTOLELHB5LBYGCGOEWTWCMCD0NMM3JWJKMDH0JSQ31JJR' # your Foursquare ID
CLIENT_SECRET = 'A33R3HGGAVZ1LGGOPDAZV45RQKTOJJCKUTVSKR2LAT1HEASA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Using the function from the lab

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
names = stadtteile['Stadtteil']
latitudes = stadtteile['Latitude']
longitudes = stadtteile['Longitude']

#df = getNearbyVenues(names,latitudes,longitudes,radius=1000)

In [12]:
#df.to_csv('venues_muc.csv')

In [35]:
df = pd.read_csv('venues_muc.csv').drop('Unnamed: 0',axis=1)

In [36]:
df.head(1)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach,48.195994,11.457013,Bäckerei Schuhmair,48.197175,11.459016,Bakery


Only use categories that occure more than 50 times

In [37]:
venue_count = df.groupby('Venue Category').size().sort_values(ascending=False)

In [38]:
df = df[df['Venue Category'].isin(venue_count[venue_count>50].index)]

Only consider Neighborhoods with 10 or more entries

In [39]:
neighborhood_count = df.groupby('Neighborhood').size().sort_values(ascending=False)

In [40]:
df = df[df['Neighborhood'].isin(neighborhood_count[neighborhood_count>=10].index)].reset_index(drop=True)

In [41]:
one_hot = pd.get_dummies(df['Venue Category'])

In [42]:
df = pd.merge(df,one_hot,left_index=True,right_index=True,how='inner')

In [43]:
df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Bakery,Bus Stop,Café,Drugstore,German Restaurant,Hotel,Ice Cream Shop,Italian Restaurant,Plaza,Restaurant,Supermarket
0,Allach,48.195994,11.457013,Bäckerei Schuhmair,48.197175,11.459016,Bakery,1,0,0,0,0,0,0,0,0,0,0
1,Allach,48.195994,11.457013,Würmtalhof,48.188834,11.460680,German Restaurant,0,0,0,0,1,0,0,0,0,0,0
2,Allach,48.195994,11.457013,Westside Hotel,48.201045,11.458564,Hotel,0,0,0,0,0,1,0,0,0,0,0
3,Allach,48.195994,11.457013,Sicilia,48.193331,11.459387,Italian Restaurant,0,0,0,0,0,0,0,1,0,0,0
4,Allach,48.195994,11.457013,dm-drogerie markt,48.194118,11.465640,Drugstore,0,0,0,1,0,0,0,0,0,0,0


In [44]:
venue_freq = df.groupby('Neighborhood').mean()

venue_freq = venue_freq.drop(venue_freq.columns.values[0:4],axis=1)

In [45]:
venue_freq.head()

,Bakery,Bus Stop,Café,Drugstore,German Restaurant,Hotel,Ice Cream Shop,Italian Restaurant,Plaza,Restaurant,Supermarket
Neighborhood,,,,,,,,,,,
Allach,0.200000,0.100000,0.000000,0.200000,0.100000,0.100000,0.000000,0.100000,0.000000,0.000000,0.200000
Altschwabing,0.075000,0.000000,0.500000,0.000000,0.100000,0.000000,0.075000,0.150000,0.075000,0.025000,0.000000
Altstadt,0.000000,0.000000,0.258065,0.032258,0.096774,0.258065,0.032258,0.032258,0.161290,0.129032,0.000000
Am Hart,0.071429,0.214286,0.142857,0.071429,0.071429,0.142857,0.000000,0.071429,0.000000,0.071429,0.142857
Au,0.000000,0.000000,0.333333,0.000000,0.111111,0.074074,0.074074,0.185185,0.185185,0.037037,0.000000


In [46]:
address = 'Munich, DE'
location = Nominatim(user_agent="Munich").geocode(address)
longitude = location.longitude
latitude = location.latitude
map_muc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, stadtteil in zip(stadtteile['Latitude'], stadtteile['Longitude'], stadtteile['Stadtteil']):
    label = '{}'.format(stadtteil)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_muc)  
    
map_muc

In [47]:
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
freq_copy = venue_freq.copy()

In [56]:
venue_freq = freq_copy.copy()

In [57]:
k = 3
cluster = KMeans(n_clusters=k).fit(venue_freq)

In [58]:
density = DBSCAN(eps=0.18, min_samples=3).fit(venue_freq)

density.labels_

array([ 0,  1, -1,  1,  1,  1, -1,  2,  2, -1, -1,  2,  0,  2,  1, -1, -1,
        2,  1,  0,  1,  1,  1,  1,  1, -1,  1,  1, -1,  0,  1,  1, -1,  1,
        1, -1, -1,  1,  1,  1, -1,  1, -1,  0,  1,  1,  1, -1],
      dtype=int64)

In [59]:
venue_freq['Cluster Label'] = cluster.labels_
venue_freq.reset_index(inplace=True)

In [60]:
stadtteile_used = stadtteile[stadtteile['Stadtteil'].isin(venue_freq['Neighborhood'].unique())]

In [61]:
venue_freq.head()

,Neighborhood,Bakery,Bus Stop,Café,Drugstore,German Restaurant,Hotel,Ice Cream Shop,Italian Restaurant,Plaza,Restaurant,Supermarket,Cluster Label
0,Allach,0.200000,0.100000,0.000000,0.200000,0.100000,0.100000,0.000000,0.100000,0.000000,0.000000,0.200000,0
1,Altschwabing,0.075000,0.000000,0.500000,0.000000,0.100000,0.000000,0.075000,0.150000,0.075000,0.025000,0.000000,2
2,Altstadt,0.000000,0.000000,0.258065,0.032258,0.096774,0.258065,0.032258,0.032258,0.161290,0.129032,0.000000,1
3,Am Hart,0.071429,0.214286,0.142857,0.071429,0.071429,0.142857,0.000000,0.071429,0.000000,0.071429,0.142857,0
4,Au,0.000000,0.000000,0.333333,0.000000,0.111111,0.074074,0.074074,0.185185,0.185185,0.037037,0.000000,2


In [62]:
k = venue_freq['Cluster Label'].max()
print(k)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.jet(np.linspace(0, 1, k+1))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stadtteile_used['Latitude'], stadtteile_used['Longitude'], venue_freq['Neighborhood'], venue_freq['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

2
